In [1]:
import pandas as pd
import os.path
import pickle
import numpy as np

In [2]:
main_dir = 'path_to_prediction_results'
output_dir = 'path_to_output_folder'

dataset = 'GSE72056'

method_list = ["SVM", "SVMrej", "RF", "LDA", "LDArej", "NMC", "kNN9",
                "ACTINN","SingleCellNet", "LAmbDA",
                "scPred", "CaSTLe","CHETAH", "scID",
                "scmapcell", "scmapcluster", "singleR", "scVI", "Cell_BLAST"]

datasetDic = {'GSE72056':'Tirosh_Melanoma',
              'GSE81861':'Li_CRC',
              'GSE75688':'Chung_BC',
              'GSE115978':'JA_Melanoma',
              'Lambrechts':'Lambrechts_LC',
              'GSE84465':'Darmanis_GBM',
              'GSE116256':'vanGalan_AML',
              'Peng':'Peng_PC'}

In [3]:
cell_list = pd.read_csv(main_dir + dataset +'/celllist.tsv', header=None)
cell_list = list(cell_list.iloc[:,0])

results = {}

with open(main_dir+dataset+'/CV_data.pkl', 'rb') as f:
    CV_data = pickle.load(f)
test_ind = CV_data['test_ind']

cell_id_order = []
for i in range(CV_data['nfolds']):
    test_ind_i = np.array(test_ind[i], dtype = 'int')
    cell_id_order += [cell_list[kk] for kk in test_ind_i]

TL = pd.read_csv(main_dir + dataset + '/' + 'SVM' + '_True_Labels.csv')

results['truth'] = list(TL.iloc[:,0])

for method in method_list:
    pred_dir = main_dir + dataset + '/' + method + '_Pred_Labels.csv'
    if os.path.isfile(pred_dir):
        PL = pd.read_csv(pred_dir)
        if method == 'LAmbDA':
            temp = list(PL.iloc[:,0])
            pred_in_order = []
            for i in range(CV_data['nfolds']):
                test_ind_i = np.array(test_ind[i], dtype = 'int')
                pred_in_order += [temp[kk] for kk in test_ind_i]
            results[method] = pred_in_order
        else:
            results[method] = list(PL.iloc[:,0])
    else:
        print(method)
        results[method] = ['UNAVAILABLE' for i in range(len(results['truth']))]

df = pd.DataFrame(results, index=cell_id_order)

#------Mark all the unlabeled cell into the same terminology-------#
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        if df.iloc[i,j] == 'unassigned' \
        or df.iloc[i,j] == 'Unassigned' \
        or df.iloc[i,j] == 'Unkno' \
        or df.iloc[i,j] == 'Unknown' \
        or df.iloc[i,j] == 'rand' \
        or df.iloc[i,j] == 'ambiguous' \
        or 'Node' in df.iloc[i,j]:
            df.iloc[i,j] = 'UNLABELED'
        
        df.iloc[i,j] = df.iloc[i,j].upper()
        df.iloc[i,j] = df.iloc[i,j].replace(' ','_')

# df.to_csv(output_dir + datasetDic[dataset] + '.tsv', sep='\t')

scPred


In [4]:
df.to_csv(output_dir + datasetDic[dataset] + '.tsv', sep='\t')